In [23]:
#importing the Libraies
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

#data collection
dataset=pd.read_csv("Social_Network_Ads.csv")
print(dataset)

#preprocess
#drop/remove user ID, model creation
dataset=dataset.drop("User ID",axis=1)
dataset

#preprocess Gender to 0 or 1
dataset=pd.get_dummies(dataset,drop_first=True).astype(int)
print(dataset.columns)

#Balanced Dataset or Imbalanced Dataset
print(dataset["Purchased"].value_counts())

#input output split
independent=dataset[['Age', 'EstimatedSalary', 'Gender_Male']]
dependent=dataset[['Purchased']]

print(independent.shape)
print(dependent.shape)

#Train Test split
from sklearn.model_selection import train_test_split
X_train,X_test,y_train,y_test=train_test_split(independent, dependent, test_size=0.30,random_state=0)

#preprocess standard Scaler
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

      User ID  Gender  Age  EstimatedSalary  Purchased
0    15624510    Male   19            19000          0
1    15810944    Male   35            20000          0
2    15668575  Female   26            43000          0
3    15603246  Female   27            57000          0
4    15804002    Male   19            76000          0
..        ...     ...  ...              ...        ...
395  15691863  Female   46            41000          1
396  15706071    Male   51            23000          1
397  15654296  Female   50            20000          1
398  15755018    Male   36            33000          0
399  15594041  Female   49            36000          1

[400 rows x 5 columns]
Index(['Age', 'EstimatedSalary', 'Purchased', 'Gender_Male'], dtype='object')
Purchased
0    257
1    143
Name: count, dtype: int64
(400, 3)
(400, 1)


In [24]:
#Model creation
#https://scikit-learn.org/stable/modules/model_evaluation.html#scoring-parameter
from sklearn.linear_model import LogisticRegression

#Model creation using Grid
from sklearn.model_selection import GridSearchCV
param_grid = {'solver':['newton-cg', 'lbfgs', 'liblinear', 'saga'],
             'penalty':['l2']} 
classifier = GridSearchCV(LogisticRegression(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted') 
# fitting the model for grid search 
classifier.fit(X_train, y_train) 

Fitting 5 folds for each of 4 candidates, totalling 20 fits


C:\Anaconda3\Lib\site-packages\sklearn\utils\validation.py:1408: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


GridSearchCV(estimator=LogisticRegression(), n_jobs=-1,
             param_grid={'penalty': ['l2'],
                         'solver': ['newton-cg', 'lbfgs', 'liblinear', 'saga']},
             scoring='f1_weighted', verbose=3)

In [25]:
# print best parameter after tuning 
#print(classifier.best_params_) 
re=classifier.cv_results_
from sklearn.metrics import f1_score
f1_macro=f1_score(y_test,y_pred,average='weighted')
print("The f1_macro value for best parameter {}:".format(classifier.best_params_),f1_macro)

The f1_macro value for best parameter {'penalty': 'l2', 'solver': 'newton-cg'}: 0.8906190214115365


In [26]:
#Evaluation Matrix: Confusion_matrix
re=classifier.cv_results_
y_pred = classifier.predict(X_test)
from sklearn.metrics import confusion_matrix
cm = confusion_matrix(y_test, y_pred)
print(cm)
#Evaluation Matrix: calculate P R F
from sklearn.metrics import classification_report
clf_report = classification_report(y_test, y_pred)
print(clf_report)

[[74  5]
 [ 8 33]]
              precision    recall  f1-score   support

           0       0.90      0.94      0.92        79
           1       0.87      0.80      0.84        41

    accuracy                           0.89       120
   macro avg       0.89      0.87      0.88       120
weighted avg       0.89      0.89      0.89       120



In [27]:
#Roc = ROC (Receiver Operating Characteristic) Curve
#AUC (Area Under the Curve)
from sklearn.metrics import roc_auc_score
roc_auc_score(y_test,classifier.predict_proba(X_test)[:,1])

np.float64(0.9481321395492436)

In [28]:
table=pd.DataFrame.from_dict(re)
table

,mean_fit_time,std_fit_time,mean_score_time,std_score_time,param_penalty,param_solver,params,split0_test_score,split1_test_score,split2_test_score,split3_test_score,split4_test_score,mean_test_score,std_test_score,rank_test_score
0,0.012949,0.001822,0.012310,0.000662,l2,newton-cg,"{'penalty': 'l2', 'solver': 'newton-cg'}",0.835985,0.802399,0.644599,0.927778,0.890114,0.820175,0.097839,1
1,0.029164,0.004563,0.008667,0.002017,l2,lbfgs,"{'penalty': 'l2', 'solver': 'lbfgs'}",0.835985,0.802399,0.644599,0.927778,0.890114,0.820175,0.097839,1
2,0.006149,0.000527,0.014076,0.003360,l2,liblinear,"{'penalty': 'l2', 'solver': 'liblinear'}",0.835985,0.802399,0.644599,0.927778,0.890114,0.820175,0.097839,1
3,0.006037,0.001507,0.012518,0.001966,l2,saga,"{'penalty': 'l2', 'solver': 'saga'}",0.835985,0.802399,0.644599,0.927778,0.890114,0.820175,0.097839,1


In [30]:
#save model
import pickle
filename="final_Sav_Model.sav"
pickle.dump(classifier,open(filename,'wb'))
load_model=pickle.load(open("final_Sav_Model.sav",'rb'))

#apply StandardScaler to user input
preinput=sc.transform([[46,41000,1]])
print(preinput)
result=load_model.predict(preinput)
result

[[ 0.79762394 -0.8330751   0.99288247]]


C:\Anaconda3\Lib\site-packages\sklearn\utils\validation.py:2739: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  warnings.warn(


array([0])